## OLAP - Python + SQL

#### Gugan S Kathiresan 

### *Task 1 -  OLAP*

Write the following queries in the university database (DDL files uploaded in repository):
1. Based on the number of total students enrolled in each course, rank the courses from
most popular to least popular. The result should include the columns: course_id,
course_title, students_num, and popularity_rank.
2. Analyze the academic performance of students across courses and departments. Show
the average grade by department, courses within that department, and overall average
grade across all departments and courses. Use the following conversions from grade
letters to numbers: [’A’,’A+’,’A-’] → 100, [’B’,’B+’,’B-’] → 90, [’C’,’C+’,’C-’] → 80,
[’D’,’D+’,’D-’] → 60, and 0 for all the other letters. The result should include the
columns: department, course, grades_average.
3. For each semester identify the top 3 students that received the highest grades in that
semester (across all the courses they took in that semester). The result should include
the columns: year, semester, student_name, grade.
Hint: First write a Common Table Expression (CTE) that uses the ROW_NUMBER()
window function to partition the grades by year and semester and assign a number to
each student based on their performance in that semester. Then, in the main query use
the row numbers assigned to the students inside a WHERE clause in order to select the
top 3 students in each partition.


In [1]:
from mysql.connector import connect, Error
from getpass import getpass
import pandas as pd

#OLAP to connect to my local mysql database
try:
    conn = connect(
        host='localhost',
        user=input('Enter username: '),
        password=getpass('Enter password: '),
        database='university'
    )
    
except Error as e:
    print(e)
    
# List of queries that satisfies the question 1 a), b), c)    
sql_queries = [
    # (a) SQL Query
        """select
        c.course_id,
        c.title as course_title,
        count(t.ID) as students_num,
        DENSE_RANK() over (order by count(t.ID) desc) as popularity_rank
    from
        course as c
    left join
        takes as t on c.course_id = t.course_id
    group by
        c.course_id, c.title
    order by
        popularity_rank;""",
    
    # (b) SQL Query
        """select
        d.dept_name as department,
        c.course_id as course,
        round(avg(
            CASE
                WHEN t.grade IN ('A', 'A+', 'A-') THEN 100
                WHEN t.grade IN ('B', 'B+', 'B-') THEN 90
                WHEN t.grade IN ('C', 'C+', 'C-') THEN 80
                WHEN t.grade IN ('D', 'D+', 'D-') THEN 60
                ELSE 0
            END
        ), 2) as grades_average
    from
        department d
    left join
        course c on d.dept_name = c.dept_name
    left join
        takes t on c.course_id = t.course_id
    group by
        d.dept_name, c.course_id
    with rollup;""", 
    
    # (c) SQL Query
        """with StudentGrades as (
        select
            t.year,
            t.semester,
            s.name AS student_name,
            t.grade,
            ROW_NUMBER() OVER (PARTITION BY t.year, t.semester order by t.grade desc) as ranking
        from
            takes t
        join
            student s on t.ID = s.ID
        )

    select
        year,
        semester,
        student_name,
        grade
    from
        StudentGrades
    where
        ranking <= 3
    order by
        year, semester, ranking;"""
]


Enter username: root
Enter password: ········


In [2]:
# 1. Based on the number of total students enrolled in each course, rank the courses from
# most popular to least popular. The result should include the columns: course_id,
# course_title, students_num, and popularity_rank.

with conn.cursor() as cursor:
    cursor.execute(sql_queries[0])
    result = cursor.fetchall()
    print("\nResults:")
    result_df1 = pd.DataFrame(result, columns = cursor.column_names) # Putting results in table

result_df1 


Results:


,course_id,course_title,students_num,popularity_rank
0,CS-101,Intro. to Computer Science,7,1
1,CS-190,Game Design,2,2
2,CS-315,Robotics,2,2
3,CS-319,Image Processing,2,2
4,CS-347,Database System Concepts,2,2
5,BIO-101,Intro. to Biology,1,3
6,BIO-301,Genetics,1,3
7,EE-181,Intro. to Digital Systems,1,3
8,FIN-201,Investment Banking,1,3
9,HIS-351,World History,1,3


In [3]:
# 2. Analyze the academic performance of students across courses and departments. Show
# the average grade by department, courses within that department, and overall average
# grade across all departments and courses. Use the following conversions from grade
# letters to numbers: [’A’,’A+’,’A-’] → 100, [’B’,’B+’,’B-’] → 90, [’C’,’C+’,’C-’] → 80,
# [’D’,’D+’,’D-’] → 60, and 0 for all the other letters. The result should include the
# columns: department, course, grades_average.

with conn.cursor() as cursor:
    cursor.execute(sql_queries[1])
    result = cursor.fetchall()
    print("\nResults:")
    result_df2 = pd.DataFrame(result, columns = cursor.column_names) # Putting results in table

print('Before handling')
result_df2

# Removing some unecessary values to make the output look good
result_df2.loc[3, 'course'] = 'Avg'
result_df2.loc[9, 'course'] = 'Avg'
result_df2.loc[11, 'course'] = 'Avg'
result_df2.loc[13, 'course'] = 'Avg'
result_df2.loc[15, 'course'] = 'Avg'
result_df2.loc[17, 'course'] = 'Avg'
result_df2.loc[19, 'course'] = 'Avg'
result_df2.loc[20, 'course'] = 'Tot Avg'
result_df2.loc[20, 'department'] = 'Tot Subjects'

print('After handling')
result_df2


Results:
Before handling
After handling


,department,course,grades_average
0,Biology,BIO-101,100.00
1,Biology,BIO-301,0.00
2,Biology,BIO-399,0.00
3,Biology,Avg,33.33
4,Comp. Sci.,CS-101,78.57
5,Comp. Sci.,CS-190,95.00
6,Comp. Sci.,CS-315,95.00
7,Comp. Sci.,CS-319,95.00
8,Comp. Sci.,CS-347,100.00
9,Comp. Sci.,Avg,88.00
